# SIADEX HTN ENGINE

imports requireds

In [1]:
!pip install ..
!pip install ../unified-planning

Processing /workspaces/up-siadex
  Preparing metadata (setup.py) ... done
  Created wheel for up-siadex: filename=up_siadex-0.0.1-py3-none-any.whl size=8662000 sha256=131a5d86bd17406b4061f78f6d710c0ea71f554148ff49cfdd652841fb2af42d
  Stored in directory: /home/codespace/.cache/pip/wheels/63/55/a1/26a2eb3a61793baa41bdcf5daf2fa5975374c48f8639a70b08
Successfully built up-siadex
  Attempting uninstall: up-siadex
    Found existing installation: up-siadex 0.0.1
    Uninstalling up-siadex-0.0.1:
      Successfully uninstalled up-siadex-0.0.1
Processing /workspaces/up-siadex/unified-planning
  Preparing metadata (setup.py) ... done
  Created wheel for unified-planning: filename=unified_planning-0.4.2-py3-none-any.whl size=352469 sha256=5b83e5c60a54a38712bfbb5804be20bcdffc0a438a019b668e02ce6589d84723
  Stored in directory: /home/codespace/.cache/pip/wheels/c4/05/f2/821c8767d2971ed17287b1008afb0607f41dad844571a5b40f
Successfully built unified-planning
  Attempting uninstall: unified-planning
  

In [3]:
from up_siadex import SIADEXEngine

import unified_planning as up
from unified_planning.shortcuts import *
from unified_planning.model.htn.hierarchical_problem import HierarchicalProblem, Task, Method
from unified_planning.io import PDDLReader
from unified_planning.io import PDDLWriter
from unified_planning.io.hpdl.hpdl_reader import HPDLReader
from unified_planning.io.hpdl.hpdl_writer import HPDLWriter
from unified_planning.engines.results import PlanGenerationResultStatus


## Registering the engine

In order to use `SIADEX`, we need to register it among the set of planning engines available for the UP library as follows.

In [4]:
env = up.environment.get_env()
env.factory.add_engine('siadex', __name__, "SIADEXEngine")

## Reading a problem from a file

In [5]:
reader = PDDLReader()
problem = reader.parse_problem("../unified-planning/unified_planning/test/pddl/htn-transport/domain.hddl", "../unified-planning/unified_planning/test/pddl/htn-transport/problem.hddl")

## Solving the problem with siadex

In [6]:
with env.factory.OneshotPlanner(name='siadex') as p:
    result = p.solve(problem)
    if result.status == PlanGenerationResultStatus.SOLVED_SATISFICING:
        print(f'{p.name()} found a valid plan!')
        print(f'The plan is: \n')
        for i,a in enumerate(result.plan.actions):
            print(f"{i}: {a}")
    else:
        print('No plan found!')

NOTE: To disable printing of planning engine credits, add this line to your code: `up.shortcuts.get_env().credits_stream = None`
  *** Credits ***
  * In operation mode `OneshotPlanner` at line 1 of `/tmp/ipykernel_14545/3527194102.py`, you are using the following planning engine:
  * Engine name: SIADEX
  * Developers:  UGR SIADEX Team
  * Description: SIADEX ENGINE



/home/codespace/.python/current/lib/python3.10/site-packages/unified_planning/engines/mixins/oneshot_planner.py:73: UserWarning: <function SIADEXEngine.name at 0x7f6c6658cb80> cannot solve this kind of problem!
  warn(msg)


SIADEX found a valid plan!
The plan is: 

0: drive(truck-0, city-loc-2, city-loc-1)
1: pick-up(truck-0, city-loc-1, package-0, capacity-0, capacity-1)
2: drive(truck-0, city-loc-1, city-loc-0)
3: drop(truck-0, city-loc-0, package-0, capacity-0, capacity-1)
4: drive(truck-0, city-loc-0, city-loc-1)
5: pick-up(truck-0, city-loc-1, package-1, capacity-0, capacity-1)
6: drive(truck-0, city-loc-1, city-loc-0)
7: drop(truck-0, city-loc-0, package-1, capacity-0, capacity-1)


In [7]:
def solve_with_siadex(problem):
    with env.factory.OneshotPlanner(name='siadex') as p:
        result = p.solve(problem)
        print(result.log_messages)
        if result.status == PlanGenerationResultStatus.SOLVED_SATISFICING:
            print(f'{p.name()} found a valid plan!')
            print(f'The plan is: \n')
            for i,a in enumerate(result.plan.actions):
                print(f"{i}: {a}")
            return result.plan
        else:
            return []
            print('No plan found!')

In [11]:
htn = HierarchicalProblem()

Location = UserType("Location")

l1 = htn.add_object("l1", Location)
l2 = htn.add_object("l2", Location)
l3 = htn.add_object("l3", Location)
l4 = htn.add_object("l4", Location)

loc = Fluent("is_on", l=Location)
htn.add_fluent(loc, default_initial_value=False)


connected = Fluent("connected", l1=Location, l2=Location)
htn.add_fluent(connected, default_initial_value=False)
htn.set_initial_value(connected(l1, l2), True)
htn.set_initial_value(connected(l2, l3), True)
htn.set_initial_value(connected(l3, l4), True)
htn.set_initial_value(connected(l4, l3), True)
htn.set_initial_value(connected(l3, l2), True)
htn.set_initial_value(connected(l2, l1), True)
htn.set_initial_value(loc(l1), True)



move = InstantaneousAction("move", l_from=Location, l_to=Location)
l_from = move.parameter("l_from")
l_to = move.parameter("l_to")
move.add_precondition(loc(l_from))
move.add_precondition(connected(l_from, l_to))
move.add_effect(loc(l_from), False)
move.add_effect(loc(l_to), True)
htn.add_action(move)

go = htn.add_task("go", target=Location)

go_noop = Method("go-noop", target=Location)
go_noop.set_task(go)
target = go_noop.parameter("target")
go_noop.add_precondition(loc(target))

htn.add_method(go_noop)

go_recursive = Method(
    "go-recursive", source=Location, inter=Location, target=Location
)

go_recursive.set_task(go, go_recursive.parameter("target"))
source = go_recursive.parameter("source")
inter = go_recursive.parameter("inter")
target = go_recursive.parameter("target")

go_recursive.add_precondition(loc(source))
go_recursive.add_precondition(connected(source, inter))

t1 = go_recursive.add_subtask(move, source, inter)
t2 = go_recursive.add_subtask(go, target)
go_recursive.set_ordered(t1, t2)
htn.add_method(go_recursive)

go1 = htn.task_network.add_subtask(go, l4)


In [13]:
solve_with_siadex(htn)

  *** Credits ***
  * In operation mode `OneshotPlanner` at line 2 of `/tmp/ipykernel_14545/937920758.py`, you are using the following planning engine:
  * Engine name: SIADEX
  * Developers:  UGR SIADEX Team
  * Description: SIADEX ENGINE

[LogMessage(level=<LogLevel.INFO: 2>, message=''), LogMessage(level=<LogLevel.ERROR: 4>, message='\n')]
SIADEX found a valid plan!
The plan is: 

0: move(l1, l2)
1: move(l2, l3)
2: move(l3, l4)


[move(l1, l2), move(l2, l3), move(l3, l4)]